In [16]:
import os
import scipy
import torch
import numpy as np
import pandas as pd
from model.classification import ResNet18, ResNet18_finetune

In [10]:
def sliding_window(signal, window, stride):
    length = signal.shape[-1]
    start = []
    i = 0
    x = []
    while i < length-window:
        x.append(signal[:, i:i+window])
        start.append(i)
        i += stride
    if start[-1] < length - window:
        x.append(signal[:, length-1024:length])
        start.append(length-1024)
    return x

In [11]:

def min_max_scaler(signal):
    min_max = pd.read_csv("./RawData/Displacement/min_max.csv").values
    new_signal = np.copy(signal)
    for i in range(signal.shape[0]):
        if len(np.unique(signal[i, :] )) != 1:
            new_signal[i, :] = (signal[i, :] - min_max[i][0])/(min_max[i][1] - min_max[i][0])
    return new_signal

In [65]:
name, _, _ = scipy.io.whosmat("./RawData/Displacement/test/test_1.mat")[0]
data = scipy.io.loadmat("./RawData/Displacement/test/test_1.mat")[name]
normalized_data = min_max_scaler(data)
data_list = sliding_window(normalized_data, window=1024, stride=128)

In [66]:
state_dict = torch.load("./Logs/Identification/classification-Displacement-synthetic-ResNet18_finetune/finetune/version_0/checkpoints/epoch=00092-val_acc=0.7759.ckpt")["state_dict"]
model = ResNet18_finetune().to("cuda").eval()
model.load_state_dict(state_dict=state_dict)

<All keys matched successfully>

In [67]:
softmax = torch.nn.Softmax(dim=1)
scenario = pd.read_csv("./RawData/Displacement/scenario.csv").values
prediction_list = []

for i in range(1, 7):
    name, _, _ = scipy.io.whosmat(f"./RawData/Displacement/test/test_{i}.mat")[0]
    data = scipy.io.loadmat(f"./RawData/Displacement/test/test_{i}.mat")[name]
    normalized_data = min_max_scaler(data)
    data_list = sliding_window(normalized_data, window=1024, stride=128)
    input = np.concatenate([data_list])
    with torch.no_grad():
        input = torch.tensor(input, dtype=torch.float32).to("cuda")
        prediction = torch.argmax(softmax(model(input)), dim=1).cpu().numpy()
        unique, count = np.unique(prediction, return_counts=True)
        predict_scenario = np.argmax(count)
        target = scenario[scenario[:, 0] == predict_scenario][0, 1:4]
        prediction_list.append(target)

In [68]:
final_prediction = np.concatenate([prediction_list])

In [71]:
np.savetxt("./prediction/identification/project3_damage_task.txt", final_prediction, fmt='%f', delimiter=",")